In [1]:
type SignalType = 
    | Buy
    | Sell
    | SignalUndefined

type PivotType = 
    | High
    | Low
    | PivotUndefined

type Candle = {
    Symbol: string
    Time: DateTime
    Close: decimal
    Open: decimal
    High: decimal
    Low: decimal
    Volume: decimal
}

type Stats = {
    Symbol: string
    Time: DateTime
    PivotType: string
    PivotPrice: decimal
    ChannelLow: decimal
    ChannelHigh: decimal
    BreakoutSignal: string
    TradeCapital: decimal
}

type CandleWithStats = {
    Candle: Candle
    Stats: Stats
}

In [2]:
#r "nuget: Dapper, 2.1.28"
#r "nuget: MathNet.Numerics, 5.0.0"
#r "nuget: System.Data.SQLite, 1.0.118"
#r "nuget: XPlot.Plotly, 4.0.6"

open XPlot.Plotly
open System
open System.Text.Json
open System.Data.SQLite
open Dapper

let databaseFilename =  "c:/users/tyueksel/Desktop/f#/reaper_f/src/Analytics/Database/analytics.sqlite"
let connectionStringFile = sprintf "Data Source=%s;Version=3;" databaseFilename  

let loadData () =

    use connection = new SQLiteConnection(connectionStringFile)
    connection.Open()

    let symbolsQuery = "SELECT * FROM symbols"
    let broadeningStatsQuery = "SELECT * FROM broadening_bottoms"

    let symbols = connection.Query<Candle>(symbolsQuery)
    let broadeningStats = connection.Query<Stats>(broadeningStatsQuery)

    let candleWithStats = 
        symbols
        |> Seq.map (fun s -> 
            let stats = broadeningStats |> Seq.find (fun b -> b.Symbol = s.Symbol && b.Time = s.Time)
            { Candle = s; Stats = stats }
        )
    candleWithStats

let selectSymbols () = 
    use connection = new SQLiteConnection(connectionStringFile)
    connection.Open()
    let symbolsQuery = "SELECT * FROM broadening_bottoms" 
    let candles = connection.Query<Stats>(symbolsQuery)
    candles

selectSymbols () |> Seq.head

    

Installed Packages Dapper, 2.1.28 MathNet.Numerics, 5.0.0 System.Data.SQLite, 1.0.118 XPlot.Plotly, 4.0.6

Error: code = CantOpen (14), message = System.Data.SQLite.SQLiteException (0x87AF03F3): unable to open database file
   at System.Data.SQLite.SQLite3.Open(String strFilename, String vfsName, SQLiteConnectionFlags connectionFlags, SQLiteOpenFlagsEnum openFlags, Int32 maxPoolSize, Boolean usePool)
   at System.Data.SQLite.SQLiteConnection.Open()
   at FSI_0006.selectSymbols()
   at <StartupCode$FSI_0006>.$FSI_0006.main@()
   at System.RuntimeMethodHandle.InvokeMethod(Object target, Void** arguments, Signature sig, Boolean isConstructor)
   at System.Reflection.MethodBaseInvoker.InvokeWithNoArgs(Object obj, BindingFlags invokeAttr)

In [ ]:
let pivotScatter(candles: CandleWithStats array) (pivotType: PivotType) : Trace = 
    let pivots = 
        candles
        |> Array.filter (fun candle -> candle.Stats.PivotType = pivotType.ToString())

    let times = 
        pivots
        |> Array.map (fun candle -> candle.Candle.Time)

    let prices = 
        pivots
        |> Array.map (fun candle -> candle.Stats.PivotPrice)

    Scatter(x = times, y = prices, mode = "markers", name = $"Pivot {pivotType}")

    

In [ ]:
let candleStickTrace (candles: CandleWithStats array) : Trace =
    Candlestick( 
        x = (candles |> Array.map (fun r -> r.Candle.Time)), 
        ``open`` = (candles |> Array.map (fun r -> r.Candle.Open)), 
        high = (candles |> Array.map (fun r -> r.Candle.High)), 
        low = (candles |> Array.map (fun r -> r.Candle.Low)), 
        close = (candles |> Array.map (fun r -> r.Candle.Close)), 
        name = "Candlestick")



In [ ]:
let channelLowTrace (candles: CandleWithStats array) : Trace =
    let channelLow = Scatter(x = (candles |> Array.map (fun r -> r.Candle.Time)),
        y = (candles |> Array.map (fun r -> r.Stats.ChannelLow)),
        mode = "lines",
        name = "Channel Low")
    channelLow


let channelHighTrace (candles: CandleWithStats array) : Trace =
    let channelHigh = Scatter(x = (candles |> Array.map (fun r -> r.Candle.Time)),
        y = (candles |> Array.map (fun r -> r.Stats.ChannelHigh)),
        mode = "lines",
        name = "Channel High")
    channelHigh

In [ ]:
let breakoutActionTrace (candles: CandleWithStats array) : Trace =
    let breakoutActions = 
        candles
        |> Array.filter (fun candle -> candle.Stats.BreakoutSignal <> SignalUndefined.ToString())

    let times = 
        breakoutActions
        |> Array.map (fun candle -> candle.Candle.Time)

    let prices = 
        breakoutActions
        |> Array.map (fun candle -> candle.Candle.Close)

    Scatter(x = times, y = prices, mode = "markers", name = "Breakout Actions")

In [ ]:

let candlesWithStats = loadData() |> Array.ofSeq
printf "%A" candlesWithStats

let candleStickTrace = candleStickTrace candlesWithStats
let breakoutActionTrace = breakoutActionTrace candlesWithStats
let pivotHighScatters = pivotScatter candlesWithStats High 
let pivotLowScatters = pivotScatter candlesWithStats Low


let candleChunks = 
    candlesWithStats
    |> Array.splitInto 3


let channelLowTraces = 
    candleChunks
    |> Array.map (fun chunk -> channelLowTrace chunk)

let channelHighTraces = 
    candleChunks
    |> Array.map (fun chunk -> channelHighTrace chunk)

let chart = Chart.Plot(
    Array.concat [| 
        [|
            candleStickTrace
            breakoutActionTrace
            pivotHighScatters
            pivotLowScatters 
        |]
        channelLowTraces
        channelHighTraces
    |])

chart.WithLayout(Layout(title = "Broadening Bottoms",
    xaxis = Xaxis(title = "Time"),
    yaxis = Yaxis(title = "Price")))

chart.Show()
// sprintf "%s" (JsonSerializer.Serialize(candlesWithStats))